In [ ]:
from google.colab import drive
import pandas as pd


# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from fastai.collab import CollabDataLoaders, collab_learner
import matplotlib.pyplot as plt
from fastai.tabular.all import RandomSplitter
from fastai.collab import *
from fastai.tabular.all import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns



In [ ]:
music_info_data = pd.read_csv('/content/drive/My Drive/Spotify_Dataset/Music_Info.csv')
listening_history_data = pd.read_csv('/content/drive/My Drive/Spotify_Dataset/User_Listening_History.csv')

In [ ]:
listening_history_merged = pd.merge(listening_history_data, music_info_data[['track_id', 'name','artist', 'genre', 'year', 'duration_ms',
                                                           'danceability', 'energy', 'loudness', 'speechiness',
                                                           'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']],
                             on='track_id', how='left')

In [ ]:
# Add a new column "song" to raw dataframe with track name, artist and year.
listening_history_merged["song"] = listening_history_merged["name"] + ", " + listening_history_merged["artist"] + ", " + listening_history_merged["year"].astype(str)


In [ ]:
listening_history_merged['playcount_normalized'] = 1

In [ ]:
def get_model_df(listening_history_merged, num_users=10000):
    # Get a unique list of all user_ids in the DataFrame
    all_users = listening_history_merged['user'].unique()
    print(all_users[:20])
    # Ensure num_users does not exceed the length of all_users
    num_users = min(num_users, len(all_users))
    print(num_users)
    # Get the first num_users from all_users
    first_users = all_users[:num_users]
    print(first_users)
    # Filter the DataFrame to include only selected users
    filtered_dataframe = listening_history_merged[listening_history_merged['user'].isin(first_users)]

    # Return the filtered DataFrame
    return filtered_dataframe

In [ ]:
listening_history_merged.rename(columns={'user_id': 'user', 'song': 'item', 'playcount': 'rating'}, inplace=True)

In [ ]:
listening_history_filtered = get_model_df(listening_history_merged, 10000)


['b80344d063b5ccb3212f76538f3d9e43d87dca9e'
 '85c1f87fea955d09b4bec2e36aee110927aedf9a'
 'bd4c6e843f00bd476847fb75c47b4fb430a06856'
 '969cc6fb74e076a68e36a04409cb9d3765757508'
 '4bd88bfb25263a75bbdd467e74018f4ae570e5df'
 'e006b1a48f466bf59feefed32bec6494495a4436'
 '9d6f0ead607ac2a6c2460e4d14fb439a146b7dec'
 '9bb911319fbc04f01755814cb5edb21df3d1a336'
 'b64cdd1a0bd907e5e00b39e345194768e330d652'
 '17aa9f6dbdf753831da8f38c71b66b64373de613'
 'd6589314c0a9bcbca4fee0c93b14bc402363afea'
 '5a905f000fc1ff3df7ca807d57edb608863db05d'
 'c737ec8c1b16ce8e39115f4432c9a7fc21ec47a1'
 '45544491ccfcdc0b0803c34f201a6287ed4e30f8'
 'ed7d4c476013b1c3dd91982b61494bf7436083ba'
 'baf47ed8da24d607e50d8684cde78b923538640f'
 '169f9f4c68b62d1887c7c0ac99d10a79cfca5daf'
 'bd8475385f0aa78830fa6dfce9e7242164b035c8'
 '0afaa5d9d04bf85af720fe8cc566a41ca3e41c97'
 '403b3b867fc71dfdcc12652f30e88bdc7ccd9aa4']
10000
['b80344d063b5ccb3212f76538f3d9e43d87dca9e'
 '85c1f87fea955d09b4bec2e36aee110927aedf9a'
 'bd4c6e843f00bd476847fb7

In [ ]:
len(listening_history_filtered)

103827

In [ ]:
listening_history_filtered.head()

,track_id,user,rating,name,artist,genre,year,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,item,playcount_normalized
0,TRIRLYL128F42539D1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,Nothing From Nothing,Billy Preston,RnB,2010,155453,0.759,0.834,-5.206,0.0276,0.58500,0.055000,0.3310,0.939,102.792,"Nothing From Nothing, Billy Preston, 2010",1
1,TRFUPBA128F934F7E1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,Paper Gangsta,Lady Gaga,Pop,2012,260000,0.823,0.530,-9.344,0.0579,0.80400,0.782000,0.1050,0.800,100.002,"Paper Gangsta, Lady Gaga, 2012",1
2,TRLQPQJ128F42AA94F,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,Soy,Gipsy Kings,NaN,2003,187106,0.797,0.592,-11.647,0.0673,0.69200,0.000002,0.1100,0.851,104.518,"Soy, Gipsy Kings, 2003",1
3,TRTUCUY128F92E1D24,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,Stacked Actors,Foo Fighters,Rock,1999,256440,0.502,0.934,-5.145,0.0500,0.00203,0.001020,0.2340,0.670,134.898,"Stacked Actors, Foo Fighters, 1999",1
4,TRHDDQG12903CB53EE,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,Heaven's Gonna Burn Your Eyes,Thievery Corporation,Electronic,2002,250360,0.516,0.337,-9.812,0.0286,0.63500,0.521000,0.0637,0.323,152.411,"Heaven's Gonna Burn Your Eyes, Thievery Corporation, 2002",1


In [ ]:
# Encode user_id and track_id to integers using .loc
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
listening_history_filtered.loc[:, 'user'] = user_encoder.fit_transform(listening_history_filtered['user'])
listening_history_filtered.loc[:, 'item'] = item_encoder.fit_transform(listening_history_filtered['track_id'])

In [ ]:
interaction_matrix = listening_history_filtered.pivot_table(index='user', columns='item', values='rating', fill_value=0)


In [ ]:
interaction_matrix

item,0,1,2,3,4,5,6,7,8,9,...,15666,15667,15668,15669,15670,15671,15672,15673,15674,15675
user,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
listening_history_filtered[200:220]

,track_id,user,rating,name,artist,genre,year,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,item,playcount_normalized
200,TRXVLCB128F42629ED,3492,2,Casio Bossa Nova,Holy Fuck,Electronic,2005,386200,0.566,0.955,-5.309,0.0849,0.000832,0.008960,0.2530,0.3640,124.471,14459,1
201,TRJRBWA128F9317175,3492,1,You Were There With Me,Four Tet,Electronic,2005,352520,0.265,0.228,-21.805,0.0472,0.741000,0.515000,0.2550,0.0341,148.073,6132,1
202,TRRYLYK128F92F89F6,3492,1,Alley Cats,Hot Chip,Electronic,2010,319818,0.734,0.526,-8.086,0.0492,0.465000,0.000251,0.1100,0.5920,125.936,10990,1
203,TRDAUST128F1472A54,3492,2,Look After Me,Hot Chip,Electronic,2006,289986,0.770,0.442,-7.676,0.0275,0.272000,0.000014,0.3200,0.6120,96.801,1973,1
204,TRSORBF128F425DE16,3492,1,Repair Machines,Vitalic,Electronic,2005,225453,0.542,0.786,-5.780,0.0968,0.001270,0.679000,0.1400,0.3480,147.105,11354,1
205,TRSVTRT12903CB450E,3492,1,The Lengths,The Black Keys,NaN,2004,293786,0.399,0.515,-5.896,0.0272,0.952000,0.001220,0.1650,0.2710,143.486,11495,1
206,TRFWGOJ128E0780C8B,3492,2,In My Place,Coldplay,Rock,2012,228346,0.423,0.566,-6.550,0.0272,0.052600,0.000004,0.1960,0.2180,144.688,3707,1
207,TRBNWKN128F425F18F,3492,1,Superheroes / Human After All / Rock'n Roll,Daft Punk,Electronic,2007,341106,0.487,0.723,-11.148,0.1430,0.000045,0.383000,0.8190,0.3910,138.007,1036,1
208,TRTJQLR128E0788AED,3492,1,Playground Love,Air,Electronic,2004,210946,0.346,0.385,-11.771,0.0288,0.233000,0.788000,0.3240,0.1400,142.997,11746,1
209,TROEGQQ12903CCD316,3492,1,Unknown Brother,The Black Keys,NaN,2010,239693,0.500,0.552,-7.888,0.0413,0.466000,0.002210,0.1080,0.2090,83.810,8622,1


In [ ]:
num_components = 50  # Number of principal components
pca = PCA(n_components=num_components)
user_pca = pca.fit_transform(interaction_matrix)
item_pca = pca.fit_transform(interaction_matrix.T)


In [ ]:
print("User PCA shape:", user_pca.shape)


User PCA shape: (10000, 50)


In [ ]:
print("Item PCA shape:", item_pca.shape)



Item PCA shape: (15676, 50)


In [ ]:
# Print the explained variance ratio for each principal component
print("Explained variance ratio:", pca.explained_variance_ratio_)



Explained variance ratio: [0.07599302 0.03526659 0.0257031  0.02419601 0.02363635 0.02008815
 0.01910692 0.01788048 0.01522954 0.01456132 0.01249301 0.00887516
 0.00861509 0.00849274 0.00834139 0.00699847 0.00661132 0.00658561
 0.00652033 0.00631012 0.0058641  0.00570614 0.00524512 0.00513281
 0.00501771 0.00484142 0.00475907 0.00432226 0.00419515 0.00383432
 0.00380246 0.00371189 0.00366981 0.00365525 0.00358153 0.00350841
 0.00348802 0.00339761 0.0033457  0.0033271  0.00331652 0.00326244
 0.00311368 0.00307318 0.00305952 0.00292687 0.00280915 0.00279126
 0.00277795 0.00273764]


In [ ]:
# Print the cumulative explained variance to understand how much variance is covered by the selected components
cumulative_explained_variance = np.cumsum(pca.explained_variance_ratio_)
print("Cumulative explained variance:", cumulative_explained_variance)



Cumulative explained variance: [0.07599302 0.11125961 0.13696271 0.16115871 0.18479506 0.20488322
 0.22399014 0.24187062 0.25710015 0.27166148 0.28415448 0.29302964
 0.30164474 0.31013748 0.31847887 0.32547734 0.33208866 0.33867427
 0.34519459 0.35150471 0.35736881 0.36307495 0.36832007 0.37345288
 0.37847059 0.38331201 0.38807108 0.39239334 0.39658849 0.40042281
 0.40422527 0.40793717 0.41160698 0.41526223 0.41884375 0.42235217
 0.42584019 0.4292378  0.4325835  0.4359106  0.43922712 0.44248956
 0.44560324 0.44867643 0.45173594 0.45466282 0.45747197 0.46026323
 0.46304118 0.46577882]


In [ ]:
# Optionally, print the principal components themselves
print("Principal components (user):")
print(user_pca)



Principal components (user):
[[-6.00522673e-01 -5.47676805e-02 -2.90490935e-02 ... -1.77575789e-02
   2.72662113e-02 -1.62804560e-02]
 [-6.00711783e-01 -5.48925000e-02 -2.91625808e-02 ... -3.16177620e-02
   6.28798244e-02 -5.33247309e-02]
 [-5.96285981e-01 -5.61381621e-02 -3.06330334e-02 ... -5.54180716e-02
   5.32755810e-01  2.68478731e-01]
 ...
 [-6.00709567e-01 -5.48189672e-02 -2.90900840e-02 ... -1.79041905e-02
   2.77785974e-02 -1.70038337e-02]
 [-6.00940537e-01 -5.48450954e-02 -2.92334966e-02 ... -2.18538587e-02
   2.92804942e-02 -1.53354520e-02]
 [ 1.39498228e+00 -5.14687945e-02 -4.32032758e-02 ... -8.70717110e-04
   6.10952138e-02 -8.21145757e-02]]


In [ ]:
print("Principal components (item):")
print(item_pca)


Principal components (item):
[[-0.05919845 -0.02578715 -0.02182775 ... -0.02200692  0.0037594
  -0.01469746]
 [-0.05893129 -0.02602454 -0.02204402 ...  0.02388263  0.0386476
  -0.03206627]
 [-0.05911748 -0.02563181 -0.02158569 ... -0.01424772  0.00709202
  -0.01618802]
 ...
 [-0.05908821 -0.02568707 -0.02169427 ... -0.02125199 -0.00036217
  -0.01372136]
 [-0.05911556 -0.02563128 -0.02158005 ... -0.01496446  0.00676243
  -0.01798264]
 [-0.05827627 -0.025753   -0.02193056 ... -0.13847188 -0.15760759
  -0.02161622]]


In [ ]:
def get_recommendations_pca(user_id, user_pca, interaction_matrix, top_n=10):
    user_index = user_encoder.transform([user_id])[0]
    user_vector = user_pca[user_index]

    # Compute similarity of the user vector with all item vectors
    similarity_scores = cosine_similarity(user_vector.reshape(1, -1), item_pca)
    similarity_scores = similarity_scores.flatten()

    # Get top N item indices based on similarity scores
    top_items_indices = similarity_scores.argsort()[-top_n:][::-1]

    # Get item IDs from indices
    recommended_items = item_encoder.inverse_transform(top_items_indices)

    return recommended_items


In [ ]:
user_id = '4e11f45d732f4861772b2906f81a7d384552ad12'
recommendations = get_recommendations_pca(user_id, user_pca, interaction_matrix, top_n=10)
print(f"Recommendations for user {user_id}: {recommendations}")

Recommendations for user 4e11f45d732f4861772b2906f81a7d384552ad12: ['TRGRZCY128F42685BA' 'TRKMHSX128E07819A5' 'TRMOTJA128E078199F'
 'TRICALF128F9310064' 'TRVZRTU128F92D54E3' 'TRRITWQ128F9311286'
 'TRGPSZD128F932E373' 'TRBDDDU128F92EAA47' 'TRJTOER128F92C5A80'
 'TRAUMIT128F424DA18']


In [ ]:
# Get unique song names with artists and years based on recommended track IDs
recommended_song_info = listening_history_merged[listening_history_merged['track_id'].isin(recommendations)][['name', 'artist', 'year']].drop_duplicates()

print("Recommended Song Names with Artist and Year:")
for index, row in recommended_song_info.iterrows():
    print(f"{row['name']} by {row['artist']} ({row['year']})")


Recommended Song Names with Artist and Year:
Dope Nose by Weezer (2002)
Tiger By My Side by Empire of the Sun (2008)
Swordfish Hotkiss Night by Empire of the Sun (2008)
Stork & Owl by TV on the Radio (2008)
Caught By The River by Doves (2010)
Live Alone by Franz Ferdinand (2009)
The Vaguest Of Feeling by Franz Ferdinand (2009)
Can't Stop Feeling by Franz Ferdinand (2013)
Opus Eclipse by Therion (1996)
N.Y. by Doves (2012)
